In [0]:
%run Workspace/Users/kumaravinash7761880@gmail.com/NYC_Taxi_Transformation/DataAccess/Mount_Point

# Data Reading

**Importing** **Libraries**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### **Reading** CSV Data

**Trip_Type Data**

In [0]:
df_trip_Type = spark.read.format("csv")\
                        .option("header", "true")\
                        .option("inferSchema", "true")\
                        .load("abfss://bronze@datalakenyctaxi.dfs.core.windows.net/trip_type/")

In [0]:
df_trip_Type.display()

**Trip_Zone Data**

In [0]:
df_trip_zone = spark.read.format("csv")\
                        .option("header", True)\
                        .option("inferSchema", True)\
                        .load("abfss://bronze@datalakenyctaxi.dfs.core.windows.net/trip_zone/")

In [0]:
df_trip_zone.display()

**trips 2024 Data**

In [0]:
myschema = '''
                vendorID BIGINT,
                lpep_pickup_datetime TIMESTAMP,
                lpep_dropoff_datetime TIMESTAMP,
                store_and_fwd_flag STRING,
                RatecodeID BIGINT,
                PULocationID BIGINT,
                DOLocationID BIGINT,
                passenger_count BIGINT,
                trip_distance DOUBLE,
                fare_amount DOUBLE,
                extra DOUBLE,
                mta_tax DOUBLE,
                tip_amount DOUBLE,
                tolls_amount DOUBLE,
                ehail_fee DOUBLE,
                improvement_surcharge DOUBLE,
                total_amount DOUBLE,
                payment_type BIGINT,
                trip_type BIGINT,
                congestion_surcharge DOUBLE
            '''

In [0]:
df_trip = spark.read.format("parquet")\
                    .option("header", True)\
                    .schema(myschema)\
                    .option("recursiveFileLookup", True)\
                    .load("abfss://bronze@datalakenyctaxi.dfs.core.windows.net/trips2024data/")

In [0]:
df_trip.display()

## **Data Transformation**

In [0]:
df_trip_Type.display()

### **Taxi-Trip **Data****

In [0]:
df_trip_Type = df_trip_Type.withColumnRenamed("description", "trips_desc")

df_trip_Type.display()

### trip_Type - Data Writing

In [0]:
df_trip_Type = df_trip_Type.write.format("parquet")\
                                    .mode("overwrite")\
                                    .save("abfss://silver@datalakenyctaxi.dfs.core.windows.net/trip_type/")

### Taxi-Trip Zone Data

In [0]:
df_trip_zone.display()

In [0]:
df_trip_zone = df_trip_zone.withColumn("Zone1", split(col("Zone"), '/')[0])\
                           .withColumn("Zone2", split(col("Zone"), '/')[1]) 
df_trip_zone.display()


In [0]:
df_trip_zone = df_trip_zone.fillna("Not Available")

In [0]:
df_trip_zone.display()

### TripZone - Data Writing

In [0]:
df_trip_zone = df_trip_zone.write.format("parquet")\
                                    .mode("overwrite")\
                                    .save("abfss://silver@datalakenyctaxi.dfs.core.windows.net/trip_zone/")


### Trips2024Data

In [0]:
df_trip.display()

In [0]:
df_trip = df_trip.withColumn("trip_pickup_year", year(col("lpep_pickup_datetime")))\
                 .withColumn("trip_pickup_month", month(col("lpep_pickup_datetime")))\
                 .withColumn("trip_pickup_date", to_date(col("lpep_pickup_datetime")))

df_trip.display()

In [0]:
df_trip = df_trip.select("VendorID", "PULocationID", "DOLocationID", "trip_distance", "fare_amount", "total_amount")

df_trip.display()

### trips2024Data - Data Writing

In [0]:
df_trip = df_trip.write.format("parquet")\
                                    .mode("overwrite")\
                                    .save("abfss://silver@datalakenyctaxi.dfs.core.windows.net/trips2024data/")